In [1]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import patsy
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)



In [2]:
!pip install ISLP
from ISLP import load_data

In [3]:
##set a random seed before beginning your analysis
np.random.seed(1)

In [4]:
##getting data insight
df = load_data("Default")

In [14]:
df['default'] = (df['default'] == 'Yes').astype('int')
df.head()

,default,student,balance,income
0,0,No,729.526495,44361.625074
1,0,Yes,817.180407,12106.134700
2,0,No,1073.549164,31767.138947
3,0,No,529.250605,35704.493935
4,0,No,785.655883,38463.495879


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  int32  
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), int32(1), object(1)
memory usage: 273.6+ KB


In [16]:
###5.a) Fit a logistic regression model that uses income and balance to predict default.

# Create a logistic regression model with a high regularization (C=1e6) and a small tolerance value
logr_model = LogisticRegression(C=10**6, tol=1e-6)
X = df[['income', 'balance']]
y = df['default']
logr_fit = logr_model.fit(X, y)
logr_coefficients = logr_fit.coef_
logistic_formula = 'default ~ income + balance'
logistic_results = sm.Logit.from_formula(formula=logistic_formula, data=df).fit()
logistic_summary = logistic_results.summary()
# Print the results
print("Logistic Regression Coefficients: {}".format(logr_coefficients))
print(logistic_summary)

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10
Logistic Regression Coefficients: [[2.08089921e-05 5.64710291e-03]]
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Wed, 01 Nov 2023   Pseudo R-squ.:                  0.4594
Time:                        22:33:25   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.5405      0.435    -26

In [17]:
##5.b Using the validation set approach, estimate the test error of this
##model. In order to do this, you must perform the following steps:
## i)Split the sample set into a training set and a validation set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [20]:
##ii. Fit a multiple logistic regression model using only the training observations.
logr_model = LogisticRegression()
logr_model.fit(X_train, y_train)
logr_model.coef_

array([[-0.00012769,  0.00049358]])

In [37]:
##iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of
## default for that individual, and classifying the individual to the default category if the posterior probability is greater
## than 0.5.
posterior_prob = logr_model.predict_proba(X_test)
default_prob = posterior_prob[:, 1]
predictions = (default_prob > 0.5).astype(int)

In [38]:
##Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.
validation_set_error = 1 - (predictions == y_test).mean()
print(f"Validation Set Error: {validation_set_error:.5f}")

Validation Set Error: 0.03220


In [42]:
## 5.C)Repeat the process in (b) three times, using three different splits
## of the observations into a training set and a validation set. Comment
## on the results obtained. 
np.random.seed(42)

validation_errors = []

for loop in range(3):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    validation_error = 1 - accuracy_score(y_val, y_pred)
    validation_errors.append(validation_error)

for i, error in enumerate(validation_errors):
    print(f'Split {i + 1} Validation Error: {error:.5f}')

Split 1 Validation Error: 0.03450
Split 2 Validation Error: 0.02300
Split 3 Validation Error: 0.03300


Split 1 Validation Error: 0.03450: This indicates that the model's performance on the first validation split is relatively good. The validation error of 0.03450 suggests that the model correctly predicts the outcome for approximately 96.55% of the cases in the first validation set. It's a low error rate, which implies a high level of accuracy. 

Split 2 Validation Error: 0.02300: The second validation error is even lower at 0.02300, indicating a very accurate model performance. In this case, the model correctly predicts the outcome for approximately 97.70% of the cases in the second validation set. This is an even better performance than in the first split.

Split 3 Validation Error: 0.03300: The third validation error is slightly higher than the second split but still relatively low at 0.03300. This implies that the model correctly predicts the outcome for around 96.70% of the cases in the third validation set. While it's slightly worse than the second split, it's still a good level of accuracy.

Overall, the results suggest that the model is performing well in terms of classification accuracy across different validation splits. The model seems to have a consistent and low error rate in each split, indicating that it generalizes well to new data. .

In [48]:
## (d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable
## for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a
## dummy variable for student leads to a reduction in the test error rate.
np.random.seed(32)
df['student_dummy'] = (df['student'] == 'Yes').astype(int)

X = df[['income', 'balance', 'student_dummy']]
y = df['default']

validation_errors_student = []

for loop in range(3):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    validation_error = 1 - accuracy_score(y_val, y_pred)
    validation_errors_student.append(validation_error)

for i, error in enumerate(validation_errors_student):
    print(f'Split {i + 1} Validation Error (with Student): {error:.5f}')


Split 1 Validation Error (with Student): 0.02800
Split 2 Validation Error (with Student): 0.03467
Split 3 Validation Error (with Student): 0.03233


In Split 1, including the "student" dummy variable reduces the error rate from 0.03450 to 0.02800. This indicates a reduction in test error when including the "student" variable.

In Split 2, including the "student" variable results in an increase in error rate from 0.02300 to 0.03467. In this case, the test error rate increases when including the "student" variable.

In Split 3, the error rate remains relatively similar when including the "student" variable (0.03300 without "student" vs. 0.03233 with "student").

The effect of including the "student" dummy variable on the test error rate is mixed. It seems to reduce the error in some splits but not in others. The impact of adding this variable can depend on the specific dataset and how it relates to the target variable ("default" in this case). It's important to consider both the statistical significance and the practical significance of including additional features in your model.

question 5.6) We continue to consider the use of a logistic regression model to
predict the probability of default using income and balance on the
Default data set. In particular, we will now compute estimates for the
standard errors of the income and balance logistic regression coefficients
in two different ways: (1) using the bootstrap, and (2) using the
standard formula for computing the standard errors in the sm.GLM()
function. Do not forget to set a random seed before beginning your
analysis.


In [52]:
##(a) Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefficients associated with
##income and balance in a multiple logistic regression model that uses both predictors.
df = load_data("Default")
np.random.seed(32)
model2 = smf.glm(formula='default ~ income + balance', data=df, family=sm.families.Binomial())
results = model2.fit()
print(results.summary())

                        Generalized Linear Model Regression Results                        
Dep. Variable:     ['default[No]', 'default[Yes]']   No. Observations:                10000
Model:                                         GLM   Df Residuals:                     9997
Model Family:                             Binomial   Df Model:                            2
Link Function:                               Logit   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:                -789.48
Date:                             Wed, 01 Nov 2023   Deviance:                       1579.0
Time:                                     23:54:38   Pearson chi2:                 6.95e+03
No. Iterations:                                  9   Pseudo R-squ. (CS):             0.1256
Covariance Type:                         nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-

the estimated standard errors for the coefficients is 4.99e-06

(b) Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs
the coefficient estimates for income and balance in the multiple
logistic regression model.

In [84]:
df = load_data("Default")
def boot_fn(data, indices):
    bootstrapped_data = data.iloc[indices]
    X = bootstrapped_data[['income', 'balance']]
    X = sm.add_constant(X)
    y = (bootstrapped_data['default'] == 'Yes').astype(int)
    model = sm.GLM(y, X, family=sm.families.Binomial())
    results = model.fit()
    coef_income = results.params[1]
    coef_balance = results.params[2]
    return [coef_income, coef_balance]
random_indices = np.random.choice(len(df), len(df), replace=True)

print('the coefficient estimates for income and balance:', boot_fn(df,random_indices))

the coefficient estimates for income and balance: [2.997799198826917e-05, 0.005880971047949065]


(c) Following the bootstrap example in the lab, use your boot_fn()
function to estimate the standard errors of the logistic regression
coefficients for income and balance.

In [85]:
## to call boot function n times we defined n as 1000
df = load_data("Default")

n = 1000
coefficient_estimates = []

for _ in range(n):
    
    random_indices = np.random.choice(len(df), len(df), replace=True)  
    
    coefficients = boot_fn(df, random_indices)
    coefficient_estimates.append(coefficients)

bootstrap_std_errors = np.std(coefficient_estimates, axis=0)


income_sd_error = bootstrap_std_errors[0]
balance_sd_error = bootstrap_std_errors[1]

print("estimated Standard Error for Income:", income_sd_error)
print("estimated Standard Error for Balance:", balance_sd_error)

estimated Standard Error for Income: 4.6819982042439346e-06
estimated Standard Error for Balance: 0.0002264350150464406


(d) Comment on the estimated standard errors obtained using the
sm.GLM() function and using the bootstrap.

Both method provides almost similar the standard errors indicating that the bootstrap method is effectively approximate the same standard errors as calculated using more traditional methods.

Still always need to consider that calculation of standard error depend on the nature of dataset.

5.7 In Sections 5.1.2 and 5.1.3, we saw that the cross_validate() function
can be used in order to compute the LOOCV test error estimate.
Alternatively, one could compute those quantities using just sm.GLM()
and the predict() method of the fitted model within a for loop. You
will now take this approach in order to compute the LOOCV error
for a simple logistic regression model on the Weekly data set. Recall
that in the context of classification problems, the LOOCV error is
given in (5.4).

In [6]:
from statsmodels.formula.api import glm
from statsmodels.genmod.families import Binomial

(a) Fit a logistic regression model that predicts Direction using Lag1
and Lag2.

In [7]:
df1 = load_data("Weekly")
df1.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [8]:
df1['Direction'] = df1['Direction'].map({'Up': 1, 'Down': 0})

model = glm('Direction ~ Lag1 + Lag2', data=df1, family=Binomial()).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Direction   No. Observations:                 1089
Model:                            GLM   Df Residuals:                     1086
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -744.11
Date:                Thu, 02 Nov 2023   Deviance:                       1488.2
Time:                        01:37:57   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007303
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2212      0.061      3.599      0.0

(b) Fit a logistic regression model that predicts Direction using Lag1
and Lag2 using all but the first observation.

In [9]:
data_excluding_first = df1.iloc[1:]

new_model = glm('Direction ~ Lag1 + Lag2', data=data_excluding_first, family=Binomial()).fit()

print(new_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:              Direction   No. Observations:                 1088
Model:                            GLM   Df Residuals:                     1085
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -743.26
Date:                Thu, 02 Nov 2023   Deviance:                       1486.5
Time:                        01:38:02   Pearson chi2:                 1.09e+03
No. Iterations:                     4   Pseudo R-squ. (CS):           0.007373
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2232      0.061      3.630      0.0

(c) Use the model from (b) to predict the direction of the first observation.
You can do this by predicting that the first observation
will go up if P(Direction = "Up"|Lag1, Lag2) > 0.5. Was this
observation correctly classified?

In [16]:
first_observation = df1.iloc[0]
lag1 = first_observation['Lag1']
lag2 = first_observation['Lag2']


first_observation_pred = new_model.predict({'Lag1': lag1, 'Lag2': lag2})


predicted_direction = 1 if first_observation_pred.values[0] > 0.5 else 0

# prediction comparision
actual_direction = first_observation['Direction']
correctly_classified = predicted_direction == actual_direction



print(f"Predicted Direction: {'Up' if predicted_direction else 'Down'}")
print(f"Actual Direction: {'Up' if actual_direction else 'Down'}")
print(f"Correctly Classified? {correctly_classified}")

Predicted Direction: Up
Actual Direction: Down
Correctly Classified? False


(d) Write a for loop from i = 1 to i = n, where n is the number of
observations in the data set, that performs each of the following
steps:

i. Fit a logistic regression model using all but the ith observation
to predict Direction using Lag1 and Lag2.
ii. Compute the posterior probability of the market moving up
for the ith observation.
iii. Use the posterior probability for the ith observation in order
to predict whether or not the market moves up.
iv. Determine whether or not an error was made in predicting
the direction for the ith observation. If an error was made,
then indicate this as a 1, and otherwise indicate it as a 0.

In [18]:
n = len(df1)
errors = []

for i in range(n):
    training_df = df1.drop(df1.index[i])
    testing_df = df1.iloc[i]
    
    # logistic regression model
    new_model2 = glm('Direction ~ Lag1 + Lag2', data=training_df, family=Binomial()).fit()
    
    # Compute the posterior probability of the market moving up for the ith observation
    predicted_prob = new_model2.predict({'Lag1': testing_df['Lag1'], 'Lag2': testing_df['Lag2']}).values[0]
    
    # Use the posterior probability for the ith observation in order to predict whether or not the market moves up
    predicted_direction = 1 if predicted_prob > 0.5 else 0
    
    # Determine whether or not an error was made in predicting the direction for the ith observation. If an error was made, then indicate this as a 1, and otherwise indicate it as a 0.
    actual_direction = testing_df['Direction']
    error = 1 if predicted_direction != actual_direction else 0
    
    errors.append(error)

print(f"Total errors: {sum(errors)}")


Total errors: 490


(e) Take the average of the n numbers obtained in (d)iv in order to
obtain the LOOCV estimate for the test error. Comment on the
results.

In [20]:
loocv_estimate_for_test_error = sum(errors) / len(df1)

print("The LOOCV estimate for the test error:", loocv_estimate_for_test_error)


The LOOCV estimate for the test error: 0.44995408631772266


LOOCV estimate is nearly 45% which is high indicating that model prediction using lag1 and lag2 are not highly accurate so its not that remarkable performance over random guessing approach.